In [1]:
import sys
sys.path.append('../datasets')

sys.path.append('../algorithms')
import torch

import matplotlib.pyplot as plt

from pansharpening import PANDataset

from torchvision import transforms
from math import sqrt

In [ ]:
# Define device (default is "cpu")
device = "cpu" 

# Define dtype
dtype = torch.float64

# Define random seed
seed = 42
torch.manual_seed(seed)

# Define data path
data_path = '/home/mhiriy/data/harvard.zarr'

In [3]:
#val_transform = transforms.Compose([transforms.ToTensor()]) # Transforms a the input data to torch tensors
dataset = PANDataset(root_dir=data_path, split='train', transform= None)

In [4]:
idx = 17
x = dataset[idx]
x.shape

(1040, 1392, 31)

In [5]:
#x = dataset[idx]
y = torch.tensor(x)
y=y.unsqueeze(0).to(device=device, dtype=dtype)

In [6]:
Y_H = x # Image multispectrale
Y_H = torch.tensor(Y_H)
Y_M = torch.sum(y, dim=3).unsqueeze(1)/y.shape[3]           # Matrice de transformation
B = dataset.process_hyperspectral_image(x,8)
B_t = dataset.process_hyperspectral_image_adjoint(B,8)                                                               # Transposée de B
R = dataset.spectral()            # Matrice de projection
B = torch.tensor(B)
B_t = torch.tensor(B_t)

Forme après flou (bande 0): (1040, 1392)
Forme après sous-échantillonnage (bande 0): (130, 174)
Forme après flou (bande 1): (1040, 1392)
Forme après sous-échantillonnage (bande 1): (130, 174)
Forme après flou (bande 2): (1040, 1392)
Forme après sous-échantillonnage (bande 2): (130, 174)
Forme après flou (bande 3): (1040, 1392)
Forme après sous-échantillonnage (bande 3): (130, 174)
Forme après flou (bande 4): (1040, 1392)
Forme après sous-échantillonnage (bande 4): (130, 174)
Forme après flou (bande 5): (1040, 1392)
Forme après sous-échantillonnage (bande 5): (130, 174)
Forme après flou (bande 6): (1040, 1392)
Forme après sous-échantillonnage (bande 6): (130, 174)
Forme après flou (bande 7): (1040, 1392)
Forme après sous-échantillonnage (bande 7): (130, 174)
Forme après flou (bande 8): (1040, 1392)
Forme après sous-échantillonnage (bande 8): (130, 174)
Forme après flou (bande 9): (1040, 1392)
Forme après sous-échantillonnage (bande 9): (130, 174)
Forme après flou (bande 10): (1040, 1392

In [7]:
# Instanciation de l'algorithme
from tv import ProximalGradient
pansharpening_model = ProximalGradient(max_iter=100, lmbda=0.1, lmbda_m=1, tau=0.1, verbose=True)

# Exécution de l'optimisation
U_result = pansharpening_model.forward(Y_H, Y_M, B, B_t, R)

  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (1040) must match the size of tensor b (130) at non-singleton dimension 0

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Image Multispectrale')
plt.imshow(Y_H.numpy())
plt.subplot(1, 2, 2)
plt.title('Image Fusionnée')
plt.imshow(U_result.detach().numpy())
plt.show()